In [7]:
import os
import json

In [8]:
def svgParse(input_directory, 
             input_file_name, 
             output_map_name, 
             colorRamp, 
             legendCaption,
             classBreaksFormat):

    input_file = os.path.join(input_directory, f'{input_file_name}')
    line_num = 0
    with open(input_file, 'rt') as fin:
        with open('temp.svg', 'wt') as fout:
            for line in fin:
                # write the stylesheet link after the first xml declaration
                if line.startswith("<?xml"):
                    fout.write(line)
                    fout.write('<?xml-stylesheet type="text/css" href="../css/styles.css" ?>\n')

                # write the script link and define the highlight pattern before the paths
                elif line.startswith("<g color-interpolation"):
                    fout.write(f'<title>{output_map_name}</title>')
                    fout.write('<script xlink:href="../js/svgScript.js" />\n')
                    fout.write('<defs>\n')
                    fout.write('<g id="background" >\n')
                    fout.write('<rect x="0"  y="0" height="5" width="5" fill="#c8c8c8" />\n')
                    fout.write('<rect x="5"  y="0" height="5" width="5" fill="#f8f8f8"/>\n')
                    fout.write('<rect x="0"  y="5" height="5" width="5" fill="#f8f8f8"/>\n')
                    fout.write('<rect x="5"  y="5" height="5" width="5" fill="#c8c8c8" />\n')
                    fout.write('</g>\n')
                    fout.write('<pattern id="OvalPattern" patternUnits="userSpaceOnUse" width="10" height="10" >\n')
                    fout.write('<use xlink:href="#background"/>\n')
                    fout.write('</pattern>\n')
                    fout.write('</defs>\n')
                    fout.write(line)

                # use the starting fill color as the class, add an onclick to enable fill change
                else:
                    for r in (('fill="#', 'class="rgb'), ('path', f'path id="feature{line_num}" fill="#ffffff" onclick="color(this.id, this.getAttribute(\'class\'))"')):
                        line = line.replace(*r)
                    fout.write(line)
                line_num += 1 

    #overwrite the original file with the new one
    os.remove(input_file)
    os.rename('temp.svg', input_file)

    # generate class break values
    p = arcpy.mp.ArcGISProject("Current")
    m = p.listMaps()[0]
    l = m.listLayers()[0]
    sym = l.symbology
    break_vals = [brk.upperBound for brk in sym.renderer.classBreaks]

    # write svg info to separate json file
    out_json = os.path.join(input_directory, f'{input_file_name}.json')
    map_dict = {
        "alt": output_map_name,
        "data": f'img/{input_file_name}',
        "colorRamp": colorRamp,
        "legendCaption": legendCaption,
        "break_vals": [classBreaksFormat.format(brk) for brk in break_vals]
    }

    with open(out_json, 'w') as json_file:
      json.dump(map_dict, json_file)

In [9]:
input_directory = r'C:\Users\Travis\Documents\ArcGIS\Projects\MappingPrize2020\MappyColoring\img'
input_file_name = 'white.svg'
output_map_name = 'White Population'
colorRamp = "OrRd"
legendCaption = "Percent of population identifying as White Alone"
classBreaksFormat = "{:.1%}"


svgParse(input_directory, 
             input_file_name, 
             output_map_name, 
             colorRamp, 
             legendCaption,
             classBreaksFormat)